# InvarLock: Policy Tier Comparison

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/invarlock/invarlock/blob/main/notebooks/invarlock_policy_tiers.ipynb)

**Purpose:** Compare Conservative/Balanced/Aggressive policy tiers and see which thresholds change.
**Runtime:** ~10–15 minutes (3 small runs)
**Requires:** `invarlock[hf]`


In [ ]:
!pip -q install "invarlock[hf]"
import sys
import invarlock

print("Python:", sys.version.split()[0])
print("InvarLock:", invarlock.__version__)


In [ ]:
!invarlock doctor --json || true


## Use a fast synthetic preset

To keep this notebook quick, we use the `synthetic` dataset provider.
For real evidence, swap to `wikitext2` or your BYOD provider.


In [ ]:
%%bash
cat > tier_preset.yaml <<'YAML'
dataset:
  provider: synthetic
  seq_len: 128
  stride: 128
  preview_n: 24
  final_n: 24
eval:
  metric:
    kind: ppl_causal
YAML


## Run three tiers


In [ ]:
%%bash
export INVARLOCK_ALLOW_NETWORK=1 INVARLOCK_TINY_RELAX=1 TRANSFORMERS_NO_TORCHVISION=1 TOKENIZERS_PARALLELISM=false
for tier in conservative balanced aggressive; do
  echo "=== tier: ${tier} ==="
  invarlock certify \
    --baseline sshleifer/tiny-gpt2 \
    --subject  sshleifer/tiny-gpt2 \
    --adapter  auto \
    --profile  dev \
    --tier     ${tier} \
    --preset   tier_preset.yaml \
    --out      runs/tier_${tier} \
    --cert-out reports/tier_${tier}
done


## Compare resolved thresholds


In [ ]:
import json
from pathlib import Path

def load(path: str) -> dict:
    return json.loads(Path(path).read_text(encoding='utf-8'))

certs = {
    tier: load(f'reports/tier_{tier}/evaluation.cert.json')
    for tier in ('conservative', 'balanced', 'aggressive')
}

for tier, cert in certs.items():
    rp = (cert.get('resolved_policy') or {})
    pm = (((rp.get('metrics') or {}).get('pm_ratio') or {}))
    print(tier, 'pm_ratio.ratio_limit_base=', pm.get('ratio_limit_base'))


## Notes

- Tier affects guard thresholds and primary-metric acceptance ranges.
- The `policy_digest` changes when tier/thresholds change.

Related docs:

- `docs/reference/tier-policy-catalog.md`
- `docs/assurance/09-tier-v1-calibration.md`
